# Imports

In [36]:
# Imports
import numpy as np
import random
# Pythonic types
from typing import Callable
import itertools
# stats
import statistics

# Introduction

## Problem 1: Generating Random Boolean Functions
- W.I.P Explain the theory in depth, talk abt defenitions of e.g. oracle and use diagrams
- reference work, notebook, websites + learning video https://www.youtube.com/watch?v=QcK0GK7DUh8&t=655s

### Problem Description
The Deutsch–Jozsa algorithm is designed to work with functions that accept a fixed number of Boolean inputs and return a single Boolean output. Each function is guaranteed to be either constant (always returns False or always returns True) or balanced (returns True for exactly half of the possible input combinations). Write a Python function random_constant_balanced that returns a randomly chosen function from the set of constant or balanced functions taking four Boolean arguments as inputs.

### 1.1 Helper function

In [37]:
def bin_args_to_int(*args: bool) -> int:
    """
    Takes an arbitrary number of Boolean (truthy/falsy) arguments 
    and converts them to a single integer using bitwise operations.
    
    Parameters:
        *args (bool): A variable number of Boolean inputs.
        
    Returns:
        int: The integer representation of the binary arguments.
    """

    # Initialize the result as 0
    result = 0

    # Iterate over each argument
    for arg in args:
        # Shift the current result left by 1 bit to make room for the new bit
        # Use bitwise OR to append the boolean value (treated as 1 or 0)
        result = (result << 1) | arg

    # After processing all arguments, 'result' will contain the combined integer value
    return result

### 1.2 Main Function: The Oracle Generator
The `random_constant_balanced` function acts as an oracle factory. Since we have $n=4$ Boolean inputs, there are $2^4 = 16$ possible input combinations. 

The function randomly decides whether to generate a constant or balanced truth table:
* **Constant:** A list of 16 `True` values or 16 `False` values.
* **Balanced:** A list containing exactly 8 `True` and 8 `False` values, shuffled randomly.

It then returns an inner function (a closure) that takes exactly four Boolean arguments, calculates the index using our helper function, and returns the corresponding value from the hidden truth table.

In [38]:
# fclosure to create a black box function that is either constant or balanced, preventing user from knowing function type. 
def random_constant_balanced() -> Callable[[bool, bool, bool, bool], bool]:
    """
    Generates a random black-box function that is either strictly constant 
    or strictly balanced for 4 Boolean inputs.
    
    Returns:
        Callable: A function that takes exactly 4 boolean arguments and 
                  returns a single boolean value.
    """

    # Decide randomly if the function will be constant or balanced
    is_constant = random.choice([True, False])

    if is_constant:
        # Constant: 16 True values OR 16 False values
        # 16 values as there are 16 possible combinations of 4 Boolean inputs (2^4 = 16)
        constant_value = random.choice([True, False])
        truth_table = [constant_value] * 16
    else:
        # Balanced: exactly 8 True values and 8 False values
        truth_table = [True] * 8 + [False] * 8
        random.shuffle(truth_table)
        
    # Defnining the inner function that takes exactly 4 Boolean arguments
    def f(b1: bool, b2: bool, b3: bool, b4: bool) -> bool:

        # Return the corresponding Boolean from our truth table
        return truth_table[bin_args_to_int(b1, b2, b3, b4)]
        
    return f

### 1.3 Demonstration
We can generate a single random function and inspect its behavior. <br>
By passing all 16 possible combinations of 4 Boolean inputs (generated via `itertools.product`) into our black-box function, we can determine its type based purely on the outputs.

In [39]:
# Generating a random function
f = random_constant_balanced()

# Generating all 16 possible 4-bit Boolean input combinations
all_inputs = list(itertools.product([False, True], repeat=4))

# Evaluating the function for all combinations
outputs = [f(*inp) for inp in all_inputs]
true_count = sum(outputs)

# Determining the type based strictly on the outputs
func_type = "Constant" if true_count in (0, 16) else "Balanced"

# Displaying the results
print(f"Generated Function Type: {func_type}")
print(f"Total True outputs:  {true_count}")
print(f"Total False outputs: {16 - true_count}\n")

# Display the formatted truth table
print(" b1 b2 b3 b4 | Output")
print("-" * 22)

# Formatting each Boolean as 0/1 for clean visual readability
for inp, out in zip(all_inputs, outputs):
    bits = "  ".join(str(int(b)) for b in inp)

    # Display the input bits and the corresponding output
    print(f"  {bits} | {out}")

Generated Function Type: Constant
Total True outputs:  16
Total False outputs: 0

 b1 b2 b3 b4 | Output
----------------------
  0  0  0  0 | True
  0  0  0  1 | True
  0  0  1  0 | True
  0  0  1  1 | True
  0  1  0  0 | True
  0  1  0  1 | True
  0  1  1  0 | True
  0  1  1  1 | True
  1  0  0  0 | True
  1  0  0  1 | True
  1  0  1  0 | True
  1  0  1  1 | True
  1  1  0  0 | True
  1  1  0  1 | True
  1  1  1  0 | True
  1  1  1  1 | True


### 1.4 Statistically Verification
To statistically verify the correctness of our function generator we will generate 1,000 random functions and assert that <br>
Every function evaluates to exactly 0, 16, or 8 `True` values across all combinations.

In [40]:
def test_random_constant_balanced(num_trials: int = 1000):
    """
    Tests the generator by creating multiple functions and validating 
    their outputs against the rules of the Deutsch-Jozsa algorithm.
    """
    all_inputs = list(itertools.product([False, True], repeat=4))
    counts = {"Constant": 0, "Balanced": 0}

    for _ in range(num_trials):
        func = random_constant_balanced()
        
        # create a list of outputs for all 16 input combinations
        outputs = [func(*inp) for inp in all_inputs]
        
        # validate that the function is either constant or balanced
        true_count = sum(outputs)
        assert true_count in (0, 8, 16), f"Invalid True count: {true_count} (Expected 0, 8, or 16)."
        
        # Tally the results
        if true_count in (0, 16):
            counts["Constant"] += 1
        else:
            counts["Balanced"] += 1

    print(f"All {num_trials} trials passed successfully.")
    print(f"Total Constant functions generated: {counts['Constant']}")
    print(f"Total Balanced functions generated: {counts['Balanced']}\n")

# Execute the test
test_random_constant_balanced()

All 1000 trials passed successfully.
Total Constant functions generated: 482
Total Balanced functions generated: 518



## Problem 2: Classical Testing for Function Type
### 2.1 problem Description
Deutsch's algorithm is designed to demonstrate a potential advantage of quantum computing over classical computation. To understand this advantage, we must first understand the classical cost of solving the underlying problem. Write a Python function determine_constant_balanced that takes as input a function f, as defined in Problem 1. The function should analyze f and return the string "constant" or "balanced" depending on whether the function is constant or balanced. Write a brief note on the efficiency of your solution. What is the maximum number of times you need to call f to be 100% certain whether it is constant or balanced?

### 2.2 The Nature of the Problem
Before implementing a solution, We must first understand the problem. <br>
For this explanation, the function ``f`` is equal to ``random_constant_balanced`` <br>
The function ``f`` operates on exactly 4 Boolean inputs, meaning its domain consists of 2^4 = 16 possible input combinations. Mathematically, this maps a 4-bit input to a 1-bit output: <br>
{0,1}^4 -> {0,1}.

Our function ``f`` is generated using a closure to hide its internal truth table from the user **(simulating a black box machine)**. <br>
We are forced to evalulate it as such, the problem guarantees that ``f`` is going to be one of the two strict categories:
1. **Constant:** The function evaluates to the exact same Boolean value (either **True** or **False** for all 16 inputs) <br>

2. **Balanced:** The function evaluates to **True** for exactly half of the inputs **8** and **False** for the remaining half (8).

**Objective:** Our objective is to determine which category ``f`` falls into by querying it the absolute minimum number of times required to reach 100% certainty.
At the end we will talk about the efficiency of the solution, which we will get by counting the number of queries the function `f` was called after determining its category as described in [The query model of computation, IBM Quantum Platform](https://quantum.cloud.ibm.com/learning/en/courses/fundamentals-of-quantum-algorithms/quantum-query-algorithms/query-model-of-computation)

### 2.3 Classical Computing Cost
To accurately determine the classical computing cost, we must evaluate both the best case and worst case scenarios when querying our black box function

Based on the [IBM Quantum Platform (The Deutsch-Jozsa algorithm)](https://quantum.cloud.ibm.com/learning/en/courses/fundamentals-of-quantum-algorithms/quantum-query-algorithms/deutsch-jozsa-algorithm) course materials, we can break down the classical cost as follows:

1. **Best-case scenario:** If we query the function twice with different inputs and receive two different outputs (e.g., one True and one False), we immediately know the function is balanced. In this ideal situation, it only takes 2 queries to reach 100% certainty.

2. **Worst-case scenario:** What if we query the function and keep getting the exact same result? Because a balanced 4-bit function contains exactly 8 identical outputs (either 8 Trues or 8 Falses), receiving 8 identical outputs in a row does not guarantee the function is constant. The very next query could yield a different result, proving it was balanced all along.

We can use the **Nielsen and Chuang's formula** to find the **maximum queries** as follows, which uses the logic we just talked about above:
$$\text{Maximum Queries} = 2^{n-1} + 1$$
$$\text{Maximum Queries Workings} = 2^{4-1} + 1 = 2^{3} + 1 = 8 + 1 = 9$$

Just like when calculating time complexity of code, just because it has a chance of doing it in linear time, the algorithm is still treated as its worst case scenario time complexity e.g. N log N. <br>
This means the cost of classical computing solving this problem will be the **Worst case**, which is always 9 queries

### 2.4 Implementation of determine_constant_balanced


In [41]:
def determine_constant_balanced(f: Callable[[bool, bool, bool, bool], bool]) -> str:
    """
    Determines if a 4-bit black-box function is constant or balanced classically.
    
    This function implements the worst-case deterministic classical query 
    limit of (2^(n-1) + 1) queries to guarantee 100% certainty.
    
    Args:
        f (Callable): The black-box function taking 4 booleans and returning 1 boolean.
        
    Returns:
        str: "constant" or "balanced".
    """
    # Generate all 16 possible Boolean combinations for a 4-bit input
    # itertools.product creates an iterator of tuples: (False, False, False, False) ... (True, True, True, True)
    # see https://docs.python.org/3/library/itertools.html#itertools.product
    possible_inputs = list(itertools.product([False, True], repeat=4))
    
    # 1st Query used to compare against all subsequent queries
    # we use the * operator to unpack the tuple of booleans into separate arguments for the function call
    first_output = f(*possible_inputs[0])
    
    # We query up to 8 more times (9 times total), which is the worst-case boundary for n=4
    # We start from index 1 since we already queried index 0
    for i in range(1, 9):
        # Query the function with the current input and fetch the output
        current_input = possible_inputs[i]
        current_output = f(*current_input)
        
        # If we ever find an output that differs from the first, it MUST be balanced
        if current_output != first_output:
            # exit early since we have definitive proof it's balanced
            return "balanced"
            
    # If we have queried 9 unique inputs and all returned the same value, 
    # based on our research, it is mathematically impossible for it to be balanced.
    return "constant"

# Example
f = random_constant_balanced()
result = determine_constant_balanced(f)
print(f"Determined function type: {result}")

Determined function type: balanced


## 2.5 Testing
While our mathematical research established that $2^{n-1} + 1 = 9$ is the worst-case query limit for a 4-bit function, a robust software engineering approach requires testing to verify this.

To prove this, I will simulate 10,000 evaluations of our random_constant_balanced oracle. By wrapping the oracle in a custom tracking class, we can intercept and count the exact number of evaluations required for every single test run without modifying the underlying algorithm.

We expect to see the following behaviors:

1. **Constant Functions:** Should always require exactly 9 queries. The algorithm cannot short-circuit because it must check $2^{n-1} + 1$ values to be 100% certain no variation exists.

2. **Balanced Functions:** 
    - **Best Case:** 2 queries (the first and second inputs yield different outputs).
    - **Worst Case:** 9 queries (the algorithm happens to check 8 identical outputs before finally finding the differing 9th output).
    - **Average Case:** We expect this to be quite low (between 2 and 3), as querying a mathematically balanced truth table (8 True, 8 False) is statistically likely to yield a difference within the first few attempts.

#### 2.51 Query Tracker Class

In [42]:
class OracleQueryTracker:
    """
    A wrapper class that behaves like the black-box function but 
    transparently counts how many times it was invoked.
    """
    def __init__(self, oracle_function):
        self.oracle_function = oracle_function
        self.query_count = 0

    def __call__(self, *args):
        # Increment the counter every time the function is evaluated
        self.query_count += 1
        # Pass the arguments to the actual oracle and return its result
        return self.oracle_function(*args)

### 2.51 Running statistical test
(W.I.P, Expand markdown explaining results and how they correlate to our orignal explanation / predicitiosn before testing)

In [43]:

# The number of simulations to run for testing
TOTAL_SIMULATIONS = 10000

# Lists to store the query counts for each type of function
constant_query_counts = []
balanced_query_counts = []

print(f"Running {TOTAL_SIMULATIONS} tests...")

for _ in range(TOTAL_SIMULATIONS):
    # 1. Generating a new random black-box function
    raw_oracle = random_constant_balanced()
    
    # 2. Wrapping it in our tracker
    tracked_oracle = OracleQueryTracker(raw_oracle)
    
    # 3. Evaluating it using our classical algorithm
    result = determine_constant_balanced(tracked_oracle)
    
    # 4. Recording the number of queries it took
    if result == "constant":
        constant_query_counts.append(tracked_oracle.query_count)
    elif result == "balanced":
        balanced_query_counts.append(tracked_oracle.query_count)

# --- Analysis and Output ---

print("\n--- Empirical Results: CONSTANT Functions ---")
print(f"Total Constant Functions Evaluated: {len(constant_query_counts)}")
print(f"Best Case (Minimum queries):  {min(constant_query_counts)}")
print(f"Worst Case (Maximum queries): {max(constant_query_counts)}")
print(f"Average Queries:              {statistics.mean(constant_query_counts):.2f}")

print("\n--- Empirical Results: BALANCED Functions ---")
print(f"Total Balanced Functions Evaluated: {len(balanced_query_counts)}")
print(f"Best Case (Minimum queries):  {min(balanced_query_counts)}")
print(f"Worst Case (Maximum queries): {max(balanced_query_counts)}")
print(f"Average Queries:              {statistics.mean(balanced_query_counts):.2f}")

Running 10000 tests...

--- Empirical Results: CONSTANT Functions ---
Total Constant Functions Evaluated: 5016
Best Case (Minimum queries):  9
Worst Case (Maximum queries): 9
Average Queries:              9.00

--- Empirical Results: BALANCED Functions ---
Total Balanced Functions Evaluated: 4984
Best Case (Minimum queries):  2
Worst Case (Maximum queries): 8
Average Queries:              2.77


## Problem 3: Quantum Oracles

## Problem 4: Deutsch's Algorithm with Qiskit

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm